In [29]:
import pandas as pd

#implementing improving search for line production

#reading the data 
df=pd.read_excel('Line Production September 2023.xlsx')


In [30]:
#constructive heuristic feasible solution

Solution1 = {'L1': ['P26', 'P6', 'P9', 'P24'],
 'L2': ['P7', 'P16', 'P27', 'P15', 'P10'],
 'L3': ['P13', 'P25'],
 'L4': ['P4', 'P8'],
 'L5': ['P21', 'P18', 'P3'],
 'L6': ['P11', 'P5', 'P23', 'P19', 'P20', 'P2', 'P14'],
 'L7': ['P22', 'P17', 'P1', 'P12']}

Solution2 = {"L1": ['P26', 'P17', 'P18', 'P20'],
"L2": ['P10', 'P7', 'P27', 'P19', 'P12'],
"L3": ['P13', 'P14', 'P22', 'P15'],
"L4": ['P11', 'P9', 'P25'],
"L5": ['P23', 'P16', 'P21'],
"L6": ['P2', 'P5', 'P8', 'P1'],
"L7": ['P6', 'P4', 'P24', 'P3']}

Solution3 = {"L1": ['P26', 'P17', 'P18', 'P20'],
"L2": ['P10', 'P7', 'P27', 'P12'],
"L3": ['P13', 'P14', 'P22', 'P15', 'P3'],
"L4": ['P11', 'P9', 'P25'],
"L5": ['P23', 'P21', 'P19'],
"L6": ['P2', 'P5', 'P8', 'P1'],
"L7": ['P6', 'P4', 'P16', 'P24']}

print(f"Solution 1 penalty costs: {calculate_penalty_cost(df, Solution1)}")
print(f"Solution 2 penalty costs: {calculate_penalty_cost(df, Solution2)}")
print(f"Solution 2 penalty costs: {calculate_penalty_cost(df, Solution3)}")

Solution 1 penalty costs: 3653
Solution 2 penalty costs: 14075
Solution 2 penalty costs: 13998


In [31]:
#penalty costs
def calculate_penalty_cost(df, solution):
    """This function calculates the penalty cost with arguments df as the data where the solution comes from,
    and solution as the solution at which we're trying to calculate the penalty cost"""
    total_penalty_cost = 0

    for line, products in solution.items():

        #initial values 
        line_penalty_cost = 0  
        current_time = 0  

        for product in products:
            #get the data of the specific product that's in the current line 
            product_row = df[df['Product'] == product]

            if not product_row.empty:

                # the products processing time on the line it's being processed on 
                processing_time = product_row[line].values[0]
                # product deadline 
                deadline = product_row['deadline'].values[0]

                # how late the product is 
                lateness = current_time + processing_time - deadline

                #check if the product is late and if yes what the cost is     
                if lateness > 0:
                    penalty_cost = lateness * product_row['penalty cost'].values[0]
                    #add the penalty cost to total cost of that line
                    line_penalty_cost += penalty_cost

                #change current time to that after processing the product 
                current_time += processing_time

        # add penalty cost of everyline together 
        total_penalty_cost += line_penalty_cost

    return total_penalty_cost

In [38]:
def improving_search(iterations: int, data: str, solution: dict):
    """This function takes a solution (solution) and the data where the solution comes from and it swaps
    out products to different lines to see if the penalty cost has improved. You do this a certain amount of times which is the iterations input"""
    import copy
    # Load data from the file
    if data.endswith('.xlsx'):
        df = pd.read_excel(data)
    elif data.endswith('.csv'):
        df = pd.read_csv(data)
    else:
        raise ValueError("Unsupported file format. Use .xlsx or .csv.")

    initial_solution = copy.deepcopy(solution)
    for i in range(iterations):
        # Create a copy of the current initial solution
        best_solution = copy.deepcopy(initial_solution)
        best_penalty_cost = calculate_penalty_cost(df, best_solution)

        #initialize to check if any improvements are made 
        improvement_made = False

        #iterate through every product in every line 
        for line, products in best_solution.items():
            for product in products:
                for line2, target_products in best_solution.items():
                    if line != line2:
                        # create copy and swap through the two copies 
                        new_solution = copy.deepcopy(best_solution)

                        #make sure product is in the first line before swapping to line2
                        if product in new_solution[line]:
                            new_solution[line].remove(product)
                            new_solution[line2].append(product)

                            #new penalty cost after swapping 
                            new_penalty_cost = calculate_penalty_cost(df, new_solution)

                            #is new penalty cost better or worse than older penalty cost
                            if new_penalty_cost < best_penalty_cost:
                                best_solution = copy.deepcopy(new_solution)
                                best_penalty_cost = new_penalty_cost
                                improvement_made = True

        #now check if improvement is made 
        if improvement_made:
            initial_solution = best_solution
        else:
            #no improvement or iterations ran out 
            break

    print("Initial Solution:")
    for line, products in solution.items():
        print(f"{line}: {', '.join(products)}")
    print("Initial Penalty Cost:", calculate_penalty_cost(df, solution))

    print("Final Best Solution:")
    for line, products in best_solution.items():
        print(f"{line}: {', '.join(products)}")
    print("Final Best Penalty Cost:", best_penalty_cost)

    return None 


In [46]:
improving_search(20, "Line Production December 2023.xlsx", Solution2)

Initial Solution:
L1: P26, P17, P18, P20
L2: P10, P7, P27, P19, P12
L3: P13, P14, P22, P15
L4: P11, P9, P25
L5: P23, P16, P21
L6: P2, P5, P8, P1
L7: P6, P4, P24, P3
Initial Penalty Cost: 26166
Final Best Solution:
L1: P5, P25
L2: P12, P2, P1, P8, P19, P16, P24
L3: P13, P14, P22, P15, P6, P4, P18, P20, P26
L4: P11, P9, P10, P7, P17
L5: P21
L6: P27, P3
L7: P23
Final Best Penalty Cost: 3320


In [37]:
import copy

# Define your initial solution
solution = {"L1": ['P26', 'P17', 'P18', 'P20'],
"L2": ['P10', 'P7', 'P27', 'P19', 'P12'],
"L3": ['P13', 'P14', 'P22', 'P15'],
"L4": ['P11', 'P9', 'P25'],
"L5": ['P23', 'P16', 'P21'],
"L6": ['P2', 'P5', 'P8', 'P1'],
"L7": ['P6', 'P4', 'P24', 'P3']}
initial_solution = copy.deepcopy(solution)

# Define the number of iterations for the improvement search
num_iterations = 1000

for i in range(num_iterations):
    # Create a copy of the current best solution
    best_solution = copy.deepcopy(initial_solution)
    best_penalty_cost = calculate_penalty_cost(df, best_solution)

    # Initialize a flag to check if any improvement is made in this iteration
    improvement_made = False

    # Iterate through each product in the current best solution
    for line, products in best_solution.items():
        for product in products:
            for line2, target_products in best_solution.items():
                if line != line2:
                    # Create a copy of the current solution to try a swap
                    new_solution = copy.deepcopy(best_solution)

                    # Check if the product is in the source line before attempting a swap
                    if product in new_solution[line]:
                        new_solution[line].remove(product)
                        new_solution[line2].append(product)

                        # Calculate the penalty cost of the new solution
                        new_penalty_cost = calculate_penalty_cost(df, new_solution)

                        # Check if the new solution has a lower penalty cost
                        if new_penalty_cost < best_penalty_cost:
                            best_solution = copy.deepcopy(new_solution)
                            best_penalty_cost = new_penalty_cost
                            improvement_made = True

    # Check if any improvement is made in this iteration
    if improvement_made:
        initial_solution = best_solution 
    else:
        # No further improvement can be made, break the loop
        break

print("Initial Solution:")
for line, products in solution.items():
    print(f"{line}: {', '.join(products)}")
print("Initial Penalty Cost:", calculate_penalty_cost(df, solution))

print("Final Best Solution:")
for line, products in best_solution.items():
    print(f"{line}: {', '.join(products)}")
print("Final Best Penalty Cost:", best_penalty_cost)


Initial Solution:
L1: P26, P17, P18, P20
L2: P10, P7, P27, P19, P12
L3: P13, P14, P22, P15
L4: P11, P9, P25
L5: P23, P16, P21
L6: P2, P5, P8, P1
L7: P6, P4, P24, P3
Initial Penalty Cost: 14075
Final Best Solution:
L1: P26, P9, P24, P1
L2: P7, P27, P10, P8
L3: P13, P22, P25, P14
L4: P11, P23, P2
L5: P16, P21, P19, P18
L6: P5, P20, P15
L7: P6, P4, P17, P12, P3
Final Best Penalty Cost: 1357
